# Purchasing tickets
## An example use case of the Digital Twin

This example just includes:
* Connecting
* Building relationships between models
* Updating values
* Modifying the relationships and updating twins
* Querying twins by relationship

In the previous scenario we made a bunch of customers. In this scenario we built a bunch of individual users 
[This is the SDK repo on Github](https://github.com/Azure/azure-sdk-for-python/tree/4559e19e2f3146a49f1eba1706bb798071f4a1f5/sdk/digitaltwins/azure-digitaltwins-core)

[Here is the doc on the query language](https://docs.microsoft.com/en-us/azure/digital-twins/concepts-query-language)


In [1]:
from azure.identity import AzureCliCredential
from azure.digitaltwins.core import DigitalTwinsClient

# using yaml instead of 
import yaml
import uuid

# using altair instead of matplotlib for vizuals
import altair as alt
from vega_datasets import data
import numpy as np
import pandas as pd


azure_cli = AzureCliCredential()
service_client = DigitalTwinsClient(
    "home-test-twin.api.wcus.digitaltwins.azure.net", azure_cli)
service_client

So from the previous notebook we uploaded some models. 

In [2]:
patron = service_client.get_model("dtmi:billmanh:patron;1")
patron.as_dict()

{'additional_properties': {}, 'display_name': {'en': 'Patron'}, 'description': {'en': 'As an example, contains all of the properties possible in the DTDL.'}, 'id': 'dtmi:billmanh:patron;1', 'upload_time': datetime.datetime(2020, 11, 19, 2, 14, 27, 210940, tzinfo=<FixedOffset '+00:00'>), 'decommissioned': False, 'model': None}


And we have a number of `cusotmers` in our ecosystem that are made from the `patron` model. 

In [38]:
query_expression = "SELECT * FROM digitaltwins t where IS_OF_MODEL('dtmi:billmanh:patron;1')"
query_result = service_client.query_twins(query_expression)

In [40]:
df_customers = pd.DataFrame([[i['$dtId'],i['satisfaction']] for i in query_result],
                           columns=['id','satisfaction'])

In [41]:
df_customers

,id,satisfaction
0,customer-cc04f3b6-39b0-4cef-bfff-a7d668cce446,10
1,customer-3cbd5e60-957d-44ff-944f-9adb42a20a52,10
2,customer-5c454e2f-f70b-4352-b75a-958f1a49beba,7
3,customer-26196fee-5ffd-457a-86b7-192a998f3cf2,9
4,customer-e6f49d8a-711b-41c3-9db8-c7ece3dbc32c,7
...,...,...
79,customer-45e9aa03-733d-4a99-b9d5-94f1c6b04214,9
80,customer-0234cb48-1fa2-43e0-b69d-36a6ff269666,9
81,customer-75b2f757-faee-4a85-bc93-e6e9ff7cd891,6
82,customer-048f85a8-173e-4305-92b8-ead2a748b07f,8


Let's add another model for `tickets`.  In our very simple model, we will assume that `customers` will buy `tickets` to our events.

In [174]:
service_client.delete_model(ticket_model_id)

In [175]:
ticket_model_json = yaml.safe_load(open("../models/ticket.json"))
service_client.create_models([ticket_model_json])

In [176]:
ticket_model_id = "dtmi:billmanh:ticket;1"
get_model = service_client.get_model(ticket_model_id)
get_model.as_dict()

{'display_name': {'en': 'ticket'},
 'description': {'en': 'an abstract ticket'},
 'id': 'dtmi:billmanh:ticket;1',
 'upload_time': '2020-11-26T17:31:20.753118Z',
 'decommissioned': False}

Note that the dict returned from the service doesn't contain all of the values that you created, but that's ok. 

In [63]:
def generate_twin(name,model_id):
    digital_twin_id = f'{name}-{str(uuid.uuid4())}'
    dt_json = {
        "$metadata": {
            "$model": model_id
        }
    }
    return digital_twin_id,dt_json

def updsert_twin():
    created_twin = service_client.upsert_digital_twin(digital_twin_id, dt_json)
    return created_twin

In [64]:
ticket_id, ticket_json = generate_twin("ticket",ticket_model_id)

In [65]:
ticket_json

{'$metadata': {'$model': 'dtmi:billmanh:ticket;1'}}

So when I create the `tickets` i'm going to create them as unsold (`available`) and for a range of events. I'm just going to introduce some tickets into the system for some shows coming up. These tickets just exist locally at this point, but we will 'go live' when we push them into the ecosystem. 

In [193]:
def generate_tickets(title,n_tickets):
    tickets = []
    for i in range(n_tickets):
        ticket_id, ticket_json = generate_twin("ticket",ticket_model_id)
        ticket_json['event_title'] = title
        ticket_json['state'] = 'open'
        ticket_json['ticket_location'] = i
        ticket_json['uid'] = f'ticket-{str(uuid.uuid4())}'
        tickets.append(ticket_json)
    return tickets

tickets_df = pd.concat([pd.DataFrame(generate_tickets('Nirvana',5)),
          pd.DataFrame(generate_tickets('Smashing Pumpkins',5)),
           pd.DataFrame(generate_tickets('Foo Fighters',5))]).reset_index(drop=True)

tickets_df

,$metadata,event_title,state,ticket_location,uid
0,{'$model': 'dtmi:billmanh:ticket;1'},Nirvana,open,0,ticket-ca0b3ba6-5653-48db-829e-3271c17dd224
1,{'$model': 'dtmi:billmanh:ticket;1'},Nirvana,open,1,ticket-5538331c-e0d6-4338-8657-7022c31abb97
2,{'$model': 'dtmi:billmanh:ticket;1'},Nirvana,open,2,ticket-09ada091-8ab9-4ca5-9586-a52fcc498ec0
3,{'$model': 'dtmi:billmanh:ticket;1'},Nirvana,open,3,ticket-4337bb3a-a556-4c57-80ca-661dd04a875d
4,{'$model': 'dtmi:billmanh:ticket;1'},Nirvana,open,4,ticket-974e491b-90d3-4191-a46a-f1c31d3d705e
5,{'$model': 'dtmi:billmanh:ticket;1'},Smashing Pumpkins,open,0,ticket-3e073eac-45fa-4415-a73f-9903ca904a6a
6,{'$model': 'dtmi:billmanh:ticket;1'},Smashing Pumpkins,open,1,ticket-91532dbb-5d3b-4dbf-bb8c-e233a9ed6912
7,{'$model': 'dtmi:billmanh:ticket;1'},Smashing Pumpkins,open,2,ticket-75407b25-27da-413d-9105-6cd1c1ba01f6
8,{'$model': 'dtmi:billmanh:ticket;1'},Smashing Pumpkins,open,3,ticket-77152f5c-2740-4bf1-9e9f-f28f4daeca16
9,{'$model': 'dtmi:billmanh:ticket;1'},Smashing Pumpkins,open,4,ticket-3d66fba8-db93-4234-817b-8cdf45d7b7ec


In [194]:
tickets_df['uid'].apply(len)

0     43
1     43
2     43
3     43
4     43
5     43
6     43
7     43
8     43
9     43
10    43
11    43
12    43
13    43
14    43
Name: uid, dtype: int64

Ok now that we know what tickets we want to sell, let's push them to the digital twin ecosystem. This is exactly the same as what we did with `Customers` in step one. 

In [195]:
ticket_twins = []
for i in tickets_df.index:
    ticket_data = tickets_df.loc[i]
    ticket_twin_id = ticket_data['uid']
    ticket_json = ticket_data.drop('uid').to_dict()
    created_twin = service_client.upsert_digital_twin(ticket_twin_id, ticket_json)
    ticket_twins.append(created_twin)

In [196]:
ticket_twins[:3]

[{'$dtId': 'ticket-ca0b3ba6-5653-48db-829e-3271c17dd224',
  '$etag': 'W/"117cef08-5877-47a4-8172-8ce828d18904"',
  'event_title': 'Nirvana',
  'state': 'open',
  'ticket_location': '0',
  '$metadata': {'$model': 'dtmi:billmanh:ticket;1',
   'event_title': {'lastUpdateTime': '2020-11-26T17:38:43.3819107Z'},
   'state': {'lastUpdateTime': '2020-11-26T17:38:43.3819107Z'},
   'ticket_location': {'lastUpdateTime': '2020-11-26T17:38:43.3819107Z'}}},
 {'$dtId': 'ticket-5538331c-e0d6-4338-8657-7022c31abb97',
  '$etag': 'W/"1be26a1f-389a-4d7a-941d-83b782913d43"',
  'event_title': 'Nirvana',
  'state': 'open',
  'ticket_location': '1',
  '$metadata': {'$model': 'dtmi:billmanh:ticket;1',
   'event_title': {'lastUpdateTime': '2020-11-26T17:38:43.5023923Z'},
   'state': {'lastUpdateTime': '2020-11-26T17:38:43.5023923Z'},
   'ticket_location': {'lastUpdateTime': '2020-11-26T17:38:43.5023923Z'}}},
 {'$dtId': 'ticket-09ada091-8ab9-4ca5-9586-a52fcc498ec0',
  '$etag': 'W/"32345192-3ac2-47d9-8f78-d3f3fae

In [190]:
# for i in tickets_df.index:
#     ticket_data = tickets_df.loc[i]
#     ticket_twin_id = ticket_data['uid']
#     service_client.delete_digital_twin(ticket_twin_id)

So now I have `customers` and I have `tickets`, but there is no relationship between them. Let's pretend that I have a website that sells tickets. When a `customer` buys a `ticket` on the website, a relationship between the two is established AND the status of the ticket changes to sold. 

* website (or mobile app) will tell the user which tickets are still available, and have features that allow them to search by row, or by concert.
* the website (or mobile app) gets the latatude and longitude of the user when they buy the tickets. 
* once purchased, the status of that ticket changes to `sold`. 


# The user experience (mobile app)

Here is where the user will buy tickets. 

This cell is a customer experience. The `customer` enters the website and looks up the available tickets for the show they want. This data comes from the website. 

In [197]:
# selection from a dropdown menu or something
selection_show = 'Nirvana'
# just grabbing the first customer. 
user = df_customers.loc[0]['id']
user_lat = np.random.randint(0,100)
user_long = np.random.randint(0,100)

query_expression = f"""
SELECT * FROM digitaltwins where IS_OF_MODEL('{ticket_model_id}') 
and state = 'open'
and event_title = '{selection_show}'
"""
query_result = service_client.query_twins(query_expression)

Note that the client dumps records after the query. Try running the cell below a few times. If you need to use the values again and again you need to put them into memory somewhere. 

In [198]:
available_tickets_df = pd.DataFrame([[i['$dtId'],i['event_title'],i['state']] for i in query_result],
                                   columns = ['$dtId','event_title','state'])
available_tickets_df

,$dtId,event_title,state
0,ticket-ca0b3ba6-5653-48db-829e-3271c17dd224,Nirvana,open
1,ticket-5538331c-e0d6-4338-8657-7022c31abb97,Nirvana,open
2,ticket-09ada091-8ab9-4ca5-9586-a52fcc498ec0,Nirvana,open
3,ticket-4337bb3a-a556-4c57-80ca-661dd04a875d,Nirvana,open
4,ticket-974e491b-90d3-4191-a46a-f1c31d3d705e,Nirvana,open


Now the user chooses to buy a ticket. The ticket state changes to closed.

In [199]:
customer_selection = available_tickets_df.loc[0]['$dtId']

service_client.get_digital_twin(customer_selection)

{'$dtId': 'ticket-ca0b3ba6-5653-48db-829e-3271c17dd224',
 '$etag': 'W/"117cef08-5877-47a4-8172-8ce828d18904"',
 'event_title': 'Nirvana',
 'state': 'open',
 'ticket_location': '0',
 '$metadata': {'$model': 'dtmi:billmanh:ticket;1',
  'event_title': {'lastUpdateTime': '2020-11-26T17:38:43.3819107Z'},
  'state': {'lastUpdateTime': '2020-11-26T17:38:43.3819107Z'},
  'ticket_location': {'lastUpdateTime': '2020-11-26T17:38:43.3819107Z'}}}

In [200]:
customer_selection = available_tickets_df.loc[0]['$dtId']

patch = [
    {
        "op": "replace",
        "path": "",
        "value": "sold"
    }
]
service_client.update_component(customer_selection,"state", patch)

**NOTE** that the path is `""` when the item is at the root of the state.

In [202]:
query_expression = f"""
SELECT * FROM digitaltwins where IS_OF_MODEL('{ticket_model_id}') 
and event_title = '{selection_show}'
"""
query_result = service_client.query_twins(query_expression)
available_tickets_df = pd.DataFrame([[i['$dtId'],i['event_title'],i['state']] for i in query_result],
                                   columns = ['$dtId','event_title','state'])
available_tickets_df

,$dtId,event_title,state
0,ticket-ca0b3ba6-5653-48db-829e-3271c17dd224,Nirvana,sold
1,ticket-5538331c-e0d6-4338-8657-7022c31abb97,Nirvana,open
2,ticket-09ada091-8ab9-4ca5-9586-a52fcc498ec0,Nirvana,open
3,ticket-4337bb3a-a556-4c57-80ca-661dd04a875d,Nirvana,open
4,ticket-974e491b-90d3-4191-a46a-f1c31d3d705e,Nirvana,open


Relationships: 
* the target is the leaf (or the customer)
* the source is the branch (or the ticket)

you can add extra stuff to the relationship. 

In [211]:
print(customer_selection)
print(user)

ticket-ca0b3ba6-5653-48db-829e-3271c17dd224
customer-cc04f3b6-39b0-4cef-bfff-a7d668cce446


In [203]:
tickethoder_relationship = {
        "$relationshipId": f"{customer_selection}ownedBy{user}",
        "$sourceId": customer_selection,
        "$relationshipName": "ownedBy",
        "$targetId": user,
        "bought_online": True
    }

service_client.upsert_relationship(
        tickethoder_relationship["$sourceId"],
        tickethoder_relationship["$relationshipId"],
        tickethoder_relationship
    )

HttpResponseError: (InvalidRelationship) Twin relationship does not align with the model. Please ensure that the relationship is in alignment with the model. See section on listing models http://aka.ms/adtv2models.